In [17]:
# pip install --user --upgrade git+https://github.com/himanshudabas/twint.git@origin/twint-fixes#egg=twint

In [18]:
import numpy as np
import pandas as pd
import twint
import time
# from datetime import datetime
from tqdm import tqdm

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', 199)
pd.options.display.float_format = '{:.2f}'.format

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

import nest_asyncio
nest_asyncio.apply()

# PATHS

In [19]:
split = 0

In [20]:
THIS_SPLIT_PATH = fr"C:/Users/mikha/Dropbox/mikhael_misc/Projects/594/Twitter-Conspiracies/Geo Cross Referencing/User Account Info Lookup/Mikhael Splits for Searching/Split {split}.csv"

# SAVE_SPLIT_PATH = fr"C:/Users/mikha/Dropbox/mikhael_misc/Projects/594/Twitter-Conspiracies/Geo Cross Referencing/User Account Info Lookup/Searched User Results/Split {split}.csv"

# Import Split List

In [21]:
local_split_users = pd.read_csv(THIS_SPLIT_PATH)

C:\Users\mikha\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (6,11,12,13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [23]:
cols_to_return = ['location', 'following', 'followers', 'likes', 'tweets', 'private', 'verified', 'join_date']

In [24]:
cols_to_overwrite = cols_to_return + ['Searched - Bio', 'Error - Bio']

In [25]:
#create new columns if they aren't there yet
if not all(x in local_split_users.columns.tolist() for x in cols_to_overwrite):
    local_split_users[cols_to_overwrite] = np.nan

# Define Lookup Function

In [27]:
def get_user_info(ID):
    try:
        c = twint.Config()
        c.User_id = ID
        c.Hide_output = True
        c.Pandas = True

        twint.run.Lookup(c)
        return twint.storage.panda.User_df[cols_to_return].values.tolist()[0]
    except:
        return [np.nan]*len(cols_to_return)

# Apply Lookup Function

In [28]:
# DELETE THIS
# local_split_users = MASTER_USER_LIST.head()

In [29]:
local_split_users = local_split_users.set_index('ID')

In [ ]:
# user_lookup_dict = {}

for i, ID in enumerate(local_split_users.index.astype(str).tolist()):
    print(i, ID)
    if i % 50 == 0:
        try:
            local_split_users.loc[int(ID), cols_to_overwrite] = get_user_info(ID) + [1, 0]
#             user_lookup_dict[ID] = get_user_info(ID)
        except:
            local_split_users.loc[int(ID), ['Searched - Bio', 'Error - Bio']] = [0, 1]
#             user_lookup_dict[ID] = [np.nan]*len(cols_to_return)
        local_split_users.to_csv(THIS_SPLIT_PATH)
    
    
    else:
        try:
            local_split_users.loc[int(ID), cols_to_overwrite] = get_user_info(ID) + [1, 0]        
        except:
            local_split_users.loc[int(ID), ['Searched - Bio', 'Error - Bio']] = [0, 1]

1732 867456964195962881
[TOR SESSION] Creating new TOR Session. Please give it a couple of seconds...
1733 2905628988
[TOR SESSION] Creating new TOR Session. Please give it a couple of seconds...
1734 125856045
[TOR SESSION] Creating new TOR Session. Please give it a couple of seconds...


# Merge Results with Split

## Create DataFrame of get_user_info() output

In [ ]:
user_information = pd.DataFrame.from_dict(user_lookup_dict, orient='index', columns=cols_to_return)

user_information = user_information.reset_index().rename(columns={'index':'ID'})

user_information['ID'] = pd.to_numeric(user_information['ID'])

## Merge DataFrame of get_user_info() output with local_split_users (on 'ID')

In [ ]:
local_split_users = local_split_users.merge(user_information, how='left', on='ID')

In [42]:
# local_split_users

,ID,Num CT Tweets - HT,Num CT Tweets - LINK,Coordinates - ELECTION 2020,State+County - ELECTION 2020,FIPS - ELECTION 2020,location,following,followers,likes,tweets,private,verified,join_date
0,948955244,9162.00,nan,NaN,NaN,nan,和光市,73.00,44.00,0.00,114906.00,False,False,2012-11-15
1,1095787080901373970,1621.00,1.00,NaN,NaN,nan,"Wisconsin, USA",143.00,129.00,5072.00,9662.00,False,False,2019-02-13
2,35301840,106.00,nan,NaN,NaN,nan,New England,5820.00,5045.00,21363.00,38887.00,False,False,2009-04-25
3,23024891,706.00,2.00,NaN,NaN,nan,USA,3742.00,3709.00,50210.00,165454.00,False,False,2009-03-06
4,3610261757,48.00,nan,NaN,NaN,nan,Here. There. There. And there.,2089.00,1144.00,472594.00,45174.00,False,False,2015-09-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51444,775349635527352320,nan,1.00,NaN,NaN,nan,NaN,nan,nan,nan,nan,NaN,NaN,NaN
51445,1304062882963845124,nan,1.00,NaN,NaN,nan,NaN,nan,nan,nan,nan,NaN,NaN,NaN
51446,2710721406,nan,1.00,NaN,NaN,nan,NaN,nan,nan,nan,nan,NaN,NaN,NaN
51447,4189005672,nan,1.00,NaN,NaN,nan,NaN,nan,nan,nan,nan,NaN,NaN,NaN


# Export Results

In [ ]:
local_split_users.to_csv(SAVE_SPLIT_PATH, index=False)

# Split Users for Lookup

In [ ]:
# MASTER_USER_LIST = pd.read_csv(r"C:/Users/mikha/Dropbox/mikhael_misc/Projects/594/Twitter-Conspiracies/Geo Cross Referencing/Master User List - GEOLOCATED.csv")

# # MASTER_USER_LIST[~MASTER_USER_LIST['FIPS - ELECTION 2020'].isnull()]['Num CT Tweets - HT'].hist(bins=100, log=True)

# MIKHAEL_SPLIT_SAVE_PATH_ROOT = r"C:/Users/mikha/Dropbox/mikhael_misc/Projects/594/Twitter-Conspiracies/Geo Cross Referencing/User Account Info Lookup/Mikhael Splits for Searching/"
# LUKE_SPLIT_SAVE_PATH_ROOT = r"C:/Users/mikha/Dropbox/mikhael_misc/Projects/594/Twitter-Conspiracies/Geo Cross Referencing/User Account Info Lookup/Lukes Splits for Searching/"

# NUM_SPLITS = 20

# for split in range(0, NUM_SPLITS):
#     if split <= 9:
#         MASTER_USER_LIST[split::NUM_SPLITS].to_csv(MIKHAEL_SPLIT_SAVE_PATH_ROOT + f'Split {split}.csv', index=False)
#     else:
#         MASTER_USER_LIST[split::NUM_SPLITS].to_csv(LUKE_SPLIT_SAVE_PATH_ROOT + f'Split {split}.csv', index=False)